In [ ]:
import sqlite3
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
fn = 'points.sqlite'
points = pd.read_sql('select * from points where not banned', sqlite3.connect(fn))
len(points[points['from_hitchwiki'] == 1.0])

In [ ]:
points.head()

In [ ]:
hitchwiki_users = points[points['from_hitchwiki'] == 1.0].nickname.unique()
len(hitchwiki_users)

In [ ]:
hitchwiki_users

In [ ]:
import requests

S = requests.Session()

URL = "https://hitchwiki.org/en/api.php"

def get_wiki_user(name):
    try:
        PARAMS = {
            "action": "query",
            "format": "json",
            "list": "users",
            "ususers": name,
            "usprop": "blockinfo|groups|editcount|registration|emailable|gender"
        }

        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()

        return DATA["query"]["users"][0]["gender"]
    except Exception as e:
        print(f"Error fetching data for user {name}: {e}")
        return None

genders = []

for u in tqdm(hitchwiki_users):
    genders.append((u, get_wiki_user(u)))

In [ ]:
users = pd.DataFrame(genders, columns=['nickname', "gender"])
users.gender.value_counts()

In [ ]:
def match_gender(row):
    if row["from_hitchwiki"] == 1.0:
        gender = users.loc[users['nickname'] == row['nickname']]["gender"].iloc[0]
        if gender == "unknown":
            return None
        return gender
    return None

In [ ]:
points["gender"] = points.apply(match_gender, axis=1)

In [ ]:
len(points[points["gender"].notna()])

In [ ]:
points.gender.value_counts()